<h2> Data </h2>

In [5]:
import pandas as pd
df = pd.read_csv("./nfl-scores-and-betting-data/spreadspoke_scores.csv")

df

,schedule_date,schedule_season,schedule_week,schedule_playoff,team_home,score_home,score_away,team_away,team_favorite_id,spread_favorite,over_under_line,stadium,stadium_neutral,weather_temperature,weather_wind_mph,weather_humidity,weather_detail
0,09/02/1966,1966,1,False,Miami Dolphins,14,23,Oakland Raiders,NaN,NaN,NaN,Orange Bowl,False,83.0,6.0,71,NaN
1,09/03/1966,1966,1,False,Houston Oilers,45,7,Denver Broncos,NaN,NaN,NaN,Rice Stadium,False,81.0,7.0,70,NaN
2,09/04/1966,1966,1,False,San Diego Chargers,27,7,Buffalo Bills,NaN,NaN,NaN,Balboa Stadium,False,70.0,7.0,82,NaN
3,09/09/1966,1966,2,False,Miami Dolphins,14,19,New York Jets,NaN,NaN,NaN,Orange Bowl,False,82.0,11.0,78,NaN
4,09/10/1966,1966,1,False,Green Bay Packers,24,3,Baltimore Colts,NaN,NaN,NaN,Lambeau Field,False,64.0,8.0,62,NaN
5,09/10/1966,1966,2,False,Houston Oilers,31,0,Oakland Raiders,NaN,NaN,NaN,Rice Stadium,False,77.0,6.0,82,NaN
6,09/10/1966,1966,2,False,San Diego Chargers,24,0,New England Patriots,NaN,NaN,NaN,Balboa Stadium,False,69.0,9.0,81,NaN
7,09/11/1966,1966,1,False,Atlanta Falcons,14,19,Los Angeles Rams,NaN,NaN,NaN,Atlanta-Fulton County Stadium,False,71.0,7.0,57,NaN
8,09/11/1966,1966,2,False,Buffalo Bills,20,42,Kansas City Chiefs,NaN,NaN,NaN,War Memorial Stadium,False,63.0,11.0,73,NaN
9,09/11/1966,1966,1,False,Detroit Lions,14,3,Chicago Bears,NaN,NaN,NaN,Tiger Stadium,False,67.0,7.0,73,NaN
